In [5]:
# Importing standard Qiskit modules
from qiskit import QuantumCircuit, QuantumRegister, IBMQ, execute, transpile
from qiskit.providers.aer import QasmSimulator
from qiskit.tools.monitor import job_monitor
from qiskit.circuit import Parameter, Instruction

# Import state tomography modules
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity
from qiskit.opflow import Zero, One, I, X, Y, Z

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import copy

In [6]:
sim = QasmSimulator()

In [7]:
test = QuantumCircuit(3)
test.x(1)
test.x(2)
test.h(2)
# prepares state: |01-> or |+z>|-z>|-x>

st_test = state_tomography_circuits(test, [0,1,2])
job = execute(st_test, sim, shots=8192)
test_res = job.result()
job = execute(st_test, sim, shots=8192)
test_res2 = job.result()
fake_res = {"data":{0:{"raw_data":[test_res, test_res2]}}}

In [49]:
import qutip as qt
g = qt.basis(2,0)
e = qt.basis(2,1)
h = qt.hadamard_transform(1)
xm = h*e
target_state1 = qt.ket2dm(qt.tensor(g,e,xm)).full()
target_state2 = qt.ket2dm(qt.tensor(xm,e,g)).full()

In [51]:
def state_tomo(result, st_qcs):
    # The expected final state; necessary to determine state tomography fidelity
    # Fit state tomography results
    tomo_fitter = StateTomographyFitter(result, st_qcs)
    rho_fit = tomo_fitter.fit(method='lstsq')
    # Compute fidelity
    return rho_fit
    # fid = state_fidelity(rho_fit, target_state)
    # return fid
    
rho = state_tomo(test_res, st_test)



state_fidelity(rho, target_state1), state_fidelity(rho, target_state2)

(0.25065742325019497, 0.9961436653538903)

In [9]:
test_r = {}
for i in range(27):
    test_r[test_res.results[i].header.name] = test_res.results[i].data.counts

In [10]:
# |01-> or |+z>|-z>|-x> 
# measure => 011 -flip-> 110 -> 6
test_r["('Z', 'Z', 'Y')"] 

{'0x6': 4014, '0x2': 4178}

In [53]:
ACTIVE_LIST = [(1,1,1), (1,1,0), (1,0,1), (0,1,1), (1,0,0), (0,1,0), (0,0,1)] # [ZXY, ZXI, ZIY, IXY, ZII, IXI, IIY]

def extract_key(key):
    # e.g. "('Z', 'Z', 'X')" -> ZZX
    return key[2] + key[7] + key[12]

def add_dicts(a,b):
    c = {}
    keys = set(list(a.keys()) + list(b.keys())) # union of keys
    for key in keys:
        c[key] = a.get(key,0) + b.get(key, 0)
    return c

def calc_parity(pauli_string, readout_string, active_spots):
    """
    Args:
        b (str): e.g. '0x6'
        active_spots (List[int]): e.g. (1, 1, 0)
    """
    n = len(pauli_string)
    
    adjusted_pauli_string = ""
    for i in range(n):
        letter = pauli_string[i]
        adjusted_pauli_string += letter if active_spots[i] else "I"
        
    
    b = list(format(int(readout_string[2:]), '#05b')[2:]) # e.g. "0x6" -> ["1", "1", "0"]
    b = b[::-1] # ["1", "1", "0"] -> ["0", "1", "1"]
    v = np.array([1-int(val)*2 for val in b]) #  ["0", "1", "1"] ->  [1, -1, -1]
    active = v*np.array(active_spots) # [1, -1, -1] * [1, 1, 0] -> [1, -1, 0]
    p = np.prod(active[active!=0]) # [1,-1] -> (1)*(-1) = -1
    y = int((1-p)/2) #map: 1,-1 -> 0,1
    return adjusted_pauli_string, y

def run_analysis(results):
    results = copy.deepcopy(results)
    
    # data map
    num_qubits = 3
    parsed_data = {} # key: e.g. "XYZ", "XYI", .. | val: for each parity measurement (e.g. <XYI>) we store [counts of 1, counts of -1], e.g. [12345, 950] 
    for num_trott_steps, result in results["data"].items():
        # data_map = {}
        reps = len(result["raw_data"])
        for i in range(3**num_qubits):  # loop over pauli strings (i.e. different tomography circuits)
            counts = {} # for each pauli string, we store total counts added together from each rep, e.g. {'0x6': 4014, '0x2': 4178}
            pauli_string = extract_key(result["raw_data"][0].results[i].header.name)
            for r in range(reps): # loop over reps
                counts = add_dicts(counts, result["raw_data"][r].results[i].data.counts) # adding counts together
            # data_map[pauli_string] = counts
            
            for active_spots in ACTIVE_LIST: # Loops through all possible parity measurements, e.g. [ZXY, ZXI, ZIY, IXY, ZII, IXI, IIY]
                for readout_string, count in counts.items(): # loops through all readout values, e.g. '0x6', '0x2'
                    adjusted_pauli_string, parity_meas = calc_parity(pauli_string, readout_string, active_spots) # ("ZXY", "0x6", (1,1,0)) -> "ZXI", 1 corresponds to <ZXI> = -1 measurement
                    if adjusted_pauli_string not in parsed_data:
                        parsed_data[adjusted_pauli_string] = [0,0] # [counts of 1, counts of -1]
                    parsed_data[adjusted_pauli_string][parity_meas] += count

        # result["data_map"] = data_map
        result["parsed_data"] = parsed_data
        
        parity = {} # key: e.g. "XYZ", "XYI", .. | val: for each parity measurement we store the expectation value (e.g. <XYI>)
        for parity_string, count in parsed_data.items():
            norm = np.sum(count)
            parity[parity_string] = (1)*count[0]/norm + (-1)*count[1]/norm # (1) * (counts of 1) + (-1)*(counts of -1) = <ZXY>
        
        result["parity"] = parity

    return results

In [24]:
calc_parity("ZZX", "0x6", (1,0,1)) # ZIZ, 011

('ZIX', 1)

In [54]:
res_analysis = run_analysis(fake_res)

In [55]:
# state prepared by the circuit above: |+z>|-z>|-x> 
a = res_analysis["data"][0]["parsed_data"]
a["ZZX"], a["ZIX"], a["IZI"], a["ZZI"], a["IZX"], a["ZII"], a["IIX"]

([16384, 0],
 [0, 49152],
 [0, 147456],
 [0, 49152],
 [49152, 0],
 [147456, 0],
 [0, 147456])

In [28]:
import qutip as qt
g = qt.basis(2,0)
e = qt.basis(2,1)

target_state_qt = qt.tensor(e,e,g)
target_state_qt = qt.ket2dm(target_state_qt)
target_state = target_state_qt.full()
target_state

array([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [31]:
pauli = {"X":qt.sigmax(),"Y":qt.sigmay(),"Z":qt.sigmaz(),"I":qt.identity(2)}
target_parity = {}
for k1, p1 in pauli.items():
    for k2, p2 in pauli.items():
        for k3, p3 in pauli.items():
            pauli_string = k1+k2+k3
            if pauli_string == "III":
                continue
            op = qt.tensor(p1,p2,p3)
            meas = (target_state_qt*op).tr()
            target_parity[pauli_string] = meas
target_parity

{'XXX': 0.0,
 'XXY': 0.0,
 'XXZ': 0.0,
 'XXI': 0.0,
 'XYX': 0.0,
 'XYY': 0.0,
 'XYZ': 0.0,
 'XYI': 0.0,
 'XZX': 0.0,
 'XZY': 0.0,
 'XZZ': 0.0,
 'XZI': 0.0,
 'XIX': 0.0,
 'XIY': 0.0,
 'XIZ': 0.0,
 'XII': 0.0,
 'YXX': 0.0,
 'YXY': 0.0,
 'YXZ': 0.0,
 'YXI': 0.0,
 'YYX': 0.0,
 'YYY': 0.0,
 'YYZ': 0.0,
 'YYI': 0.0,
 'YZX': 0.0,
 'YZY': 0.0,
 'YZZ': 0.0,
 'YZI': 0.0,
 'YIX': 0.0,
 'YIY': 0.0,
 'YIZ': 0.0,
 'YII': 0.0,
 'ZXX': 0.0,
 'ZXY': 0.0,
 'ZXZ': 0.0,
 'ZXI': 0.0,
 'ZYX': 0.0,
 'ZYY': 0.0,
 'ZYZ': 0.0,
 'ZYI': 0.0,
 'ZZX': 0.0,
 'ZZY': 0.0,
 'ZZZ': 1.0,
 'ZZI': 1.0,
 'ZIX': 0.0,
 'ZIY': 0.0,
 'ZIZ': -1.0,
 'ZII': -1.0,
 'IXX': 0.0,
 'IXY': 0.0,
 'IXZ': 0.0,
 'IXI': 0.0,
 'IYX': 0.0,
 'IYY': 0.0,
 'IYZ': 0.0,
 'IYI': 0.0,
 'IZX': 0.0,
 'IZY': 0.0,
 'IZZ': -1.0,
 'IZI': -1.0,
 'IIX': 0.0,
 'IIY': 0.0,
 'IIZ': 1.0}